In [1]:
import numpy as np
import pandas as pd
import datetime
import math
from os import listdir
from os.path import isfile, join
import json

In [2]:
out='../output/'
inp='../input/monthly/'
pref='dam'

In [3]:
dat=pd.DataFrame(columns=[0,1,2])
onlyfiles = [f for f in listdir(inp) if isfile(join(inp, f))]
for f in onlyfiles:
    if (f.split('_')[0]==pref):
        #print(f)
        with open(join(inp, f), "r") as read_file:
            data = pd.DataFrame(json.load(read_file))
            dat=pd.concat([dat,data])

dat

,0,1,2
0,Date:Time,Price(USD/MWh),Turnover(MWh)
1,(2016/01/01) 00-01,40.1,477.5
2,(2016/01/01) 01-02,40.08,486.6
3,(2016/01/01) 02-03,40.06,495.6
4,(2016/01/01) 03-04,40.08,496.6
...,...,...,...
192,(2021/07/08) 23-24,55.09,160.2
193,Min,35.02,43
194,Max,297.75,197.7
195,Average,112.52473958333333333333333333,113.1875


In [4]:
dat.drop(dat[dat[0] == 'Date:Time'].index, inplace = True)
dat.drop(dat[dat[0] == 'Min'].index, inplace = True)
dat.drop(dat[dat[0] == 'Max'].index, inplace = True)
dat.drop(dat[dat[0] == 'Average'].index, inplace = True)
dat.drop(dat[dat[0] == 'Total'].index, inplace = True)
dat.columns=['datehour','price','volume']
dat['price']=dat['price'].astype(float)
dat['volume']=dat['volume'].astype(float)
dat

,datehour,price,volume
1,(2016/01/01) 00-01,40.10,477.5
2,(2016/01/01) 01-02,40.08,486.6
3,(2016/01/01) 02-03,40.06,495.6
4,(2016/01/01) 03-04,40.08,496.6
5,(2016/01/01) 04-05,40.07,502.9
...,...,...,...
188,(2021/07/08) 19-20,276.17,98.6
189,(2021/07/08) 20-21,250.19,121.9
190,(2021/07/08) 21-22,110.93,165.6
191,(2021/07/08) 22-23,57.71,143.0


In [5]:
dat['date']=pd.to_datetime(dat.datehour.str.slice(1,11),format="%Y/%m/%d")
dat['hour']=dat.datehour.str.slice(13,15).astype(int)
dat

,datehour,price,volume,date,hour
1,(2016/01/01) 00-01,40.10,477.5,2016-01-01,0
2,(2016/01/01) 01-02,40.08,486.6,2016-01-01,1
3,(2016/01/01) 02-03,40.06,495.6,2016-01-01,2
4,(2016/01/01) 03-04,40.08,496.6,2016-01-01,3
5,(2016/01/01) 04-05,40.07,502.9,2016-01-01,4
...,...,...,...,...,...
188,(2021/07/08) 19-20,276.17,98.6,2021-07-08,19
189,(2021/07/08) 20-21,250.19,121.9,2021-07-08,20
190,(2021/07/08) 21-22,110.93,165.6,2021-07-08,21
191,(2021/07/08) 22-23,57.71,143.0,2021-07-08,22


In [6]:
dat=dat.drop(['datehour'],1)
dat

,price,volume,date,hour
1,40.10,477.5,2016-01-01,0
2,40.08,486.6,2016-01-01,1
3,40.06,495.6,2016-01-01,2
4,40.08,496.6,2016-01-01,3
5,40.07,502.9,2016-01-01,4
...,...,...,...,...
188,276.17,98.6,2021-07-08,19
189,250.19,121.9,2021-07-08,20
190,110.93,165.6,2021-07-08,21
191,57.71,143.0,2021-07-08,22


In [7]:
hourly=dat.set_index(pd.DatetimeIndex(dat['date']))
hourly['year']=hourly.index.year
hourly['month']=hourly.index.month
hourly['day']=hourly.index.day
hourly['day_of_week']=hourly.index.dayofweek
hourly=hourly.drop(['date'],axis=1)
hourly

,price,volume,hour,year,month,day,day_of_week
date,,,,,,,
2016-01-01,40.10,477.5,0,2016,1,1,4
2016-01-01,40.08,486.6,1,2016,1,1,4
2016-01-01,40.06,495.6,2,2016,1,1,4
2016-01-01,40.08,496.6,3,2016,1,1,4
2016-01-01,40.07,502.9,4,2016,1,1,4
...,...,...,...,...,...,...,...
2021-07-08,276.17,98.6,19,2021,7,8,3
2021-07-08,250.19,121.9,20,2021,7,8,3
2021-07-08,110.93,165.6,21,2021,7,8,3


In [8]:
hourly['value']=hourly['price']*hourly['volume']
daily=hourly.groupby(['year','month','day']).sum().drop(['hour'],axis=1)
daily['price_weight_av']=daily['value']/daily['volume']
daily=daily.drop(['price'], axis=1)
daily['price_max']=hourly[['year','month','day','price']].groupby(['year','month','day']).max()
daily['price_min']=hourly[['year','month','day','price']].groupby(['year','month','day']).min()
daily['price_mean']=hourly[['year','month','day','price']].groupby(['year','month','day']).mean()

daily['power_max']=hourly[['year','month','day','volume']].groupby(['year','month','day']).max()
daily['power_min']=hourly[['year','month','day','volume']].groupby(['year','month','day']).min()
daily['power_mean']=hourly[['year','month','day','volume']].groupby(['year','month','day']).mean()

daily['price_morning_max']=hourly.loc[hourly['hour']<11][['year','month','day','price']].groupby(['year','month','day']).max()
daily['price_evening_max']=hourly.loc[hourly['hour']>=11][['year','month','day','price']].groupby(['year','month','day']).max()
daily['price_standard']=hourly.loc[(hourly['hour']>=12) & (hourly['hour']<=15)][['year','month','day','price']].groupby(['year','month','day']).mean()

daily

volume  day_of_week       value  price_weight_av  price_max  \
year month day                                                                
2016 1     1    7873.8           96  336882.508        42.785251      60.08   
           2    7319.5          120  345982.021        47.268532     118.49   
           3    7198.7          144  351712.733        48.857812     118.47   
           4    4457.0            0  225938.810        50.693024     177.66   
           5    3750.4           24  179068.882        47.746609     181.40   
...                ...          ...         ...              ...        ...   
2021 7     4    3161.0          144  145427.490        46.006798      82.94   
           5    2473.8            0  293965.736       118.831650     285.07   
           6    2347.1           24  277224.250       118.113523     288.13   
           7    2653.0           48  336557.750       126.859310     284.20   
           8    2718.8           72  341759.162       125.702208     290.07   

                price_min  price_mean  power_max  power_min  power_mean  \
year month day                                                            
2016 1     1        35.09   45.982500      502.9       53.0  328.075000   
           2        40.03   54.114167      486.0       40.0  304.979167   
           3        44.06   54.959167      486.0       40.0  299.945833   
           4        46.70   80.164583      486.0        0.0  185.708333   
           5        42.06   84.503333      436.0        0.0  156.266667   
...                   ...         ...        ...        ...         ...   
2021 7     4        35.02   46.445000      185.0       54.0  131.708333   
           5        55.05  129.912917      161.0       43.0  103.075000   
           6        55.05  129.688333      150.0       59.9   97.795833   
           7        55.06  129.812500      166.7       71.8  110.541667   
           8        55.05  130.756667      183.3       64.1  113.283333   

                price_morning_max  price_evening_max  price_standard  
year month day                                                        
2016 1     1                48.03              60.08         45.8275  
           2                56.05             118.49         50.2275  
           3                56.01             118.47         50.6050  
           4                80.02             177.66         80.0200  
           5               148.40             181.40         80.0500  
...                           ...                ...             ...  
2021 7     4                54.95              82.94         44.6600  
           5               193.06             285.07        110.0250  
           6               193.15             288.13        109.0775  
           7               193.63             284.20        108.5525  
           8               196.84             290.07        108.4825  

[2016 rows x 13 columns]

In [9]:
hourly['cat']='off'

if (pref=='dam'):
    hourly.loc[(hourly['day_of_week']<5) & (hourly['hour']==6),'cat']='standard'
    hourly.loc[(hourly['day_of_week']<5) & (hourly['hour']>=7) & (hourly['hour']<=9),'cat']='morning'

    hourly.loc[(hourly['day_of_week']<5) & (hourly['hour']>=10) & (hourly['hour']<=16),'cat']='standard'

    hourly.loc[(hourly['day_of_week']<5) & (hourly['hour']==17) & (hourly['month']>=6) & (hourly['month']<=8),'cat']='evening'
    hourly.loc[(hourly['day_of_week']<5) & (hourly['hour']==17) & ((hourly['month']<=5) | (hourly['month']>=9)),'cat']='standard'
    hourly.loc[(hourly['day_of_week']<5) & (hourly['hour']==18) & (hourly['month']<=3),'cat']='standard'
    hourly.loc[(hourly['day_of_week']<5) & (hourly['hour']==18) & (hourly['month']>=4),'cat']='evening'
    hourly.loc[(hourly['day_of_week']<5) & (hourly['hour']==19),'cat']='evening'
    hourly.loc[(hourly['day_of_week']<5) & (hourly['hour']==20) & (hourly['month']<=3),'cat']='evening'
    hourly.loc[(hourly['day_of_week']<5) & (hourly['hour']==20) & (hourly['month']>=4),'cat']='standard'
    hourly.loc[(hourly['day_of_week']<5) & (hourly['hour']==21),'cat']='standard'

    hourly.loc[(hourly['day_of_week']==5) & (hourly['hour']>=7) & (hourly['hour']<=11),'cat']='standard'
    hourly.loc[(hourly['day_of_week']==5) & (hourly['hour']==17) & (hourly['month']>=6) & (hourly['month']<=8),'cat']='standard'
    hourly.loc[(hourly['day_of_week']==5) & (hourly['hour']==18) & (hourly['month']>=4),'cat']='standard'
    hourly.loc[(hourly['day_of_week']==5) & (hourly['hour']==19),'cat']='standard'
    hourly.loc[(hourly['day_of_week']==5) & (hourly['hour']==20) & (hourly['month']<=3),'cat']='standard'

if (pref=='fpm'):
    hourly.loc[(hourly['day_of_week']<5) & (hourly['hour']>=6) & (hourly['hour']<=21),'cat']='standard'
    hourly.loc[(hourly['day_of_week']==5) & (hourly['hour']>=7) & (hourly['hour']<=11),'cat']='standard'
    hourly.loc[(hourly['day_of_week']==5) & (hourly['hour']>=18) & (hourly['hour']<=19),'cat']='standard'

if (pref=='fpw'):
    hourly.loc[(hourly['day_of_week']<5) & (hourly['hour']==5),'cat']='standard'
    hourly.loc[(hourly['day_of_week']<5) & (hourly['hour']>=6) & (hourly['hour']<=8),'cat']='morning'
    hourly.loc[(hourly['day_of_week']<5) & (hourly['hour']>=9) & (hourly['hour']<=17),'cat']='standard'
    hourly.loc[(hourly['day_of_week']<5) & (hourly['hour']>=18) & (hourly['hour']<=19),'cat']='evening'
    
    hourly.loc[(hourly['day_of_week']==5) & (hourly['hour']>=7) & (hourly['hour']<=11),'cat']='standard'
    hourly.loc[(hourly['day_of_week']==5) & (hourly['hour']>=18) & (hourly['hour']<=19),'cat']='standard'
    
hourly

,price,volume,hour,year,month,day,day_of_week,value,cat
date,,,,,,,,,
2016-01-01,40.10,477.5,0,2016,1,1,4,19147.750,off
2016-01-01,40.08,486.6,1,2016,1,1,4,19502.928,off
2016-01-01,40.06,495.6,2,2016,1,1,4,19853.736,off
2016-01-01,40.08,496.6,3,2016,1,1,4,19903.728,off
2016-01-01,40.07,502.9,4,2016,1,1,4,20151.203,off
...,...,...,...,...,...,...,...,...,...
2021-07-08,276.17,98.6,19,2021,7,8,3,27230.362,evening
2021-07-08,250.19,121.9,20,2021,7,8,3,30498.161,standard
2021-07-08,110.93,165.6,21,2021,7,8,3,18370.008,standard


In [10]:
daily['off_price_mean']=hourly.loc[hourly['cat']=='off'].reset_index()[['year','month','day','price']].groupby(['year','month','day']).mean()
daily['standard_price_mean']=hourly.loc[hourly['cat']=='standard'].reset_index()[['year','month','day','price']].groupby(['year','month','day']).mean()
daily['morning_price_mean']=hourly.loc[hourly['cat']=='morning'].reset_index()[['year','month','day','price']].groupby(['year','month','day']).mean()
daily['evening_price_mean']=hourly.loc[hourly['cat']=='evening'].reset_index()[['year','month','day','price']].groupby(['year','month','day']).mean()

daily['off_hour_count']=hourly.loc[hourly['cat']=='off'].reset_index()[['year','month','day','price']].groupby(['year','month','day']).count()
daily['standard_hour_count']=hourly.loc[hourly['cat']=='standard'].reset_index()[['year','month','day','price']].groupby(['year','month','day']).count()
daily['morning_hour_count']=hourly.loc[hourly['cat']=='morning'].reset_index()[['year','month','day','price']].groupby(['year','month','day']).count()
daily['evening_hour_count']=hourly.loc[hourly['cat']=='evening'].reset_index()[['year','month','day','price']].groupby(['year','month','day']).count()

daily['off_volume']=hourly.loc[hourly['cat']=='off'].reset_index()[['year','month','day','volume']].groupby(['year','month','day']).sum()
daily['standard_volume']=hourly.loc[hourly['cat']=='standard'].reset_index()[['year','month','day','volume']].groupby(['year','month','day']).sum()
daily['morning_volume']=hourly.loc[hourly['cat']=='morning'].reset_index()[['year','month','day','volume']].groupby(['year','month','day']).sum()
daily['evening_volume']=hourly.loc[hourly['cat']=='evening'].reset_index()[['year','month','day','volume']].groupby(['year','month','day']).sum()

daily['off_value']=hourly.loc[hourly['cat']=='off'].reset_index()[['year','month','day','value']].groupby(['year','month','day']).sum()
daily['standard_value']=hourly.loc[hourly['cat']=='standard'].reset_index()[['year','month','day','value']].groupby(['year','month','day']).sum()
daily['morning_value']=hourly.loc[hourly['cat']=='morning'].reset_index()[['year','month','day','value']].groupby(['year','month','day']).sum()
daily['evening_value']=hourly.loc[hourly['cat']=='evening'].reset_index()[['year','month','day','value']].groupby(['year','month','day']).sum()

daily

volume  day_of_week       value  price_weight_av  price_max  \
year month day                                                                
2016 1     1    7873.8           96  336882.508        42.785251      60.08   
           2    7319.5          120  345982.021        47.268532     118.49   
           3    7198.7          144  351712.733        48.857812     118.47   
           4    4457.0            0  225938.810        50.693024     177.66   
           5    3750.4           24  179068.882        47.746609     181.40   
...                ...          ...         ...              ...        ...   
2021 7     4    3161.0          144  145427.490        46.006798      82.94   
           5    2473.8            0  293965.736       118.831650     285.07   
           6    2347.1           24  277224.250       118.113523     288.13   
           7    2653.0           48  336557.750       126.859310     284.20   
           8    2718.8           72  341759.162       125.702208     290.07   

                price_min  price_mean  power_max  power_min  power_mean  ...  \
year month day                                                           ...   
2016 1     1        35.09   45.982500      502.9       53.0  328.075000  ...   
           2        40.03   54.114167      486.0       40.0  304.979167  ...   
           3        44.06   54.959167      486.0       40.0  299.945833  ...   
           4        46.70   80.164583      486.0        0.0  185.708333  ...   
           5        42.06   84.503333      436.0        0.0  156.266667  ...   
...                   ...         ...        ...        ...         ...  ...   
2021 7     4        35.02   46.445000      185.0       54.0  131.708333  ...   
           5        55.05  129.912917      161.0       43.0  103.075000  ...   
           6        55.05  129.688333      150.0       59.9   97.795833  ...   
           7        55.06  129.812500      166.7       71.8  110.541667  ...   
           8        55.05  130.756667      183.3       64.1  113.283333  ...   

                morning_hour_count  evening_hour_count  off_volume  \
year month day                                                       
2016 1     1                   3.0                 2.0      3851.5   
           2                   NaN                 NaN      5969.5   
           3                   NaN                 NaN      7198.7   
           4                   3.0                 2.0      3828.0   
           5                   3.0                 2.0      3230.0   
...                            ...                 ...         ...   
2021 7     4                   NaN                 NaN      3161.0   
           5                   3.0                 3.0       924.0   
           6                   3.0                 3.0       954.3   
           7                   3.0                 3.0      1000.5   
           8                   3.0                 3.0       993.2   

                standard_volume  morning_volume  evening_volume   off_value  \
year month day                                                                
2016 1     1             2920.3           993.0           109.0  152306.418   
           2             1350.0             NaN             NaN  271567.021   
           3                NaN             NaN             NaN  351712.733   
           4              464.0           109.0            56.0  179880.680   
           5              368.4           109.0            43.0  136366.420   
...                         ...             ...             ...         ...   
2021 7     4                NaN             NaN             NaN  145427.490   
           5             1023.9           348.0           177.9   51073.308   
           6              893.9           294.0           204.9   52595.742   
           7              974.6           403.7           274.2   55332.987   
           8             1111.2           373.7           240.7   55078.719   

        

In [11]:
daily.to_csv(out + pref + '_daily.csv')

In [12]:
hourly.to_csv(out + pref + '_hourly.csv')

In [13]:
hourly_flat=hourly.reset_index().set_index(['date'])
hourly_slim=hourly_flat.loc[(hourly_flat['year']<2021)][['hour','price','volume','cat']]
hourly_slim

,hour,price,volume,cat
date,,,,
2016-01-01,0,40.10,477.5,off
2016-01-01,1,40.08,486.6,off
2016-01-01,2,40.06,495.6,off
2016-01-01,3,40.08,496.6,off
2016-01-01,4,40.07,502.9,off
...,...,...,...,...
2020-12-31,19,160.03,60.0,evening
2020-12-31,20,60.06,40.0,standard
2020-12-31,21,60.08,64.0,standard


In [14]:
hourly_slim.to_csv(out + pref + '_hourly_slim.csv')

In [15]:
monthly=daily.groupby(['year','month']).mean().reset_index(['month','year'])
monthly=monthly.drop(['morning_volume','off_volume','standard_volume','evening_volume','price_weight_av'],1)
monthly.rename(columns={'value':'value_mean','volume':'volume_mean','off_hour_count':'off_hour_mean','standard_hour_count':'standard_hour_mean','morning_hour_count':'morning_hour_mean','evening_hour_count':'evening_hour_mean'}, inplace=True)
monthly[['price_max','standard_price_max','off_price_max','morning_price_max','evening_price_max']]=daily.groupby(['year','month']).max().reset_index(['month','year'])[['price_mean','standard_price_mean','off_price_mean','morning_price_mean','evening_price_mean']]
monthly[['price_min','standard_price_min','off_price_min','morning_price_min','evening_price_min']]=daily.groupby(['year','month']).min().reset_index(['month','year'])[['price_mean','standard_price_mean','off_price_mean','morning_price_mean','evening_price_mean']]
monthly[['price_stdev','standard_price_stdev','off_price_stdev','morning_price_stdev','evening_price_stdev']]=daily.groupby(['year','month']).std().reset_index(['month','year'])[['price_mean','standard_price_mean','off_price_mean','morning_price_mean','evening_price_mean']]

monthly[['volume','morning_volume','evening_volume','standard_volume','off_volume']]=daily.groupby(['year','month']).sum().reset_index(['month','year'])[['volume','morning_volume','evening_volume','standard_volume','off_volume']]
monthly[['value','morning_value','evening_value','standard_value','off_value']]=daily.groupby(['year','month']).sum().reset_index(['month','year'])[['value','morning_value','evening_value','standard_value','off_value']]

monthly[['off_hour_count','standard_hour_count', 'morning_hour_count', 'evening_hour_count']]=daily.groupby(['year','month']).sum().reset_index(['month','year'])[['off_hour_count','standard_hour_count', 'morning_hour_count', 'evening_hour_count']]

monthly['price_weight_mean']=monthly['value'] / monthly['volume']
monthly['off_price_weight_mean']=monthly['off_value'] / monthly['off_volume']
monthly['standard_price_weight_mean']=monthly['standard_value'] / monthly['standard_volume']
monthly['morning_price_weight_mean']=monthly['morning_value'] / monthly['morning_volume']
monthly['evening_price_weight_mean']=monthly['evening_value'] / monthly['evening_volume']


monthly['off_power']=monthly['off_volume'] / monthly['off_hour_count']
monthly['morning_power']=monthly['morning_volume'] / monthly['morning_hour_count']
monthly['standard_power']=monthly['standard_volume'] / monthly['standard_hour_count']
monthly['evening_power']=monthly['evening_volume'] / monthly['evening_hour_count']

monthly

,year,month,volume_mean,day_of_week,value_mean,price_max,price_min,price_mean,power_max,power_min,...,evening_hour_count,price_weight_mean,off_price_weight_mean,standard_price_weight_mean,morning_price_weight_mean,evening_price_weight_mean,off_power,morning_power,standard_power,evening_power
0,2016,1,3717.709677,74.064516,208682.461355,106.684000,45.982500,82.592661,362.651613,17.606452,...,42.0,56.131995,50.454284,81.002858,77.850006,137.720998,270.376190,33.409524,56.583459,37.350000
1,2016,2,3833.444828,69.379310,259149.828966,123.798500,75.023333,92.095937,367.334483,23.231034,...,42.0,67.602337,56.682217,106.547628,129.026815,156.801247,274.616358,12.280952,75.799614,42.580952
2,2016,3,2859.545161,68.645161,185387.936968,108.651500,60.115833,89.230239,285.867742,18.232258,...,46.0,64.831267,55.360656,98.717668,116.099471,107.341031,210.933133,15.502899,56.748043,34.786957
3,2016,4,1887.733333,72.533333,117227.480300,114.641667,42.853500,85.739122,183.693333,7.236667,...,42.0,62.099598,57.215878,104.192200,104.410448,139.262082,150.996736,7.793651,19.055263,4.438095
4,2016,5,1587.229032,69.548387,89668.599613,105.676500,28.659583,79.727871,165.183871,4.661290,...,44.0,56.493800,49.868152,106.019186,95.368696,140.331232,124.639943,10.831818,18.262963,4.170455
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
62,2021,3,2718.232258,66.838710,157367.627194,78.673750,27.023333,61.705841,189.854839,58.119355,...,46.0,57.893370,40.688997,61.365324,107.069933,131.375021,120.432530,87.672464,122.960142,80.008696
63,2021,4,2679.000000,71.333333,160762.373667,110.314000,30.873750,81.261906,185.530000,40.316667,...,44.0,60.008351,37.204868,66.760284,142.181426,252.474165,133.637195,74.968182,109.424815,46.463636
64,2021,5,2483.148387,71.354839,167652.750194,113.129583,34.269167,84.040032,162.409677,48.145161,...,42.0,67.516203,37.192371,90.791796,145.617597,285.751219,130.257703,81.076190,87.254511,51.383333
65,2021,6,2393.520000,68.933333,257923.627300,158.559000,52.754167,122.022061,183.256667,33.226667,...,66.0,107.759128,56.401495,131.411501,197.932285,289.100821,114.964815,124.809091,90.660317,52.624242


In [16]:
if (pref=='dam'):
    monthly['evening_days']=daily.loc[daily['evening_hour_count']>0].groupby(['year','month']).count().reset_index(['month','year'])['volume']
    monthly['morning_days']=daily.loc[daily['morning_hour_count']>0].groupby(['year','month']).count().reset_index(['month','year'])['volume']
    monthly['days']=daily.groupby(['year','month']).count().reset_index(['month','year'])['volume']

monthly

,year,month,volume_mean,day_of_week,value_mean,price_max,price_min,price_mean,power_max,power_min,...,standard_price_weight_mean,morning_price_weight_mean,evening_price_weight_mean,off_power,morning_power,standard_power,evening_power,evening_days,morning_days,days
0,2016,1,3717.709677,74.064516,208682.461355,106.684000,45.982500,82.592661,362.651613,17.606452,...,81.002858,77.850006,137.720998,270.376190,33.409524,56.583459,37.350000,21,21,31
1,2016,2,3833.444828,69.379310,259149.828966,123.798500,75.023333,92.095937,367.334483,23.231034,...,106.547628,129.026815,156.801247,274.616358,12.280952,75.799614,42.580952,21,21,29
2,2016,3,2859.545161,68.645161,185387.936968,108.651500,60.115833,89.230239,285.867742,18.232258,...,98.717668,116.099471,107.341031,210.933133,15.502899,56.748043,34.786957,23,23,31
3,2016,4,1887.733333,72.533333,117227.480300,114.641667,42.853500,85.739122,183.693333,7.236667,...,104.192200,104.410448,139.262082,150.996736,7.793651,19.055263,4.438095,21,21,30
4,2016,5,1587.229032,69.548387,89668.599613,105.676500,28.659583,79.727871,165.183871,4.661290,...,106.019186,95.368696,140.331232,124.639943,10.831818,18.262963,4.170455,22,22,31
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
62,2021,3,2718.232258,66.838710,157367.627194,78.673750,27.023333,61.705841,189.854839,58.119355,...,61.365324,107.069933,131.375021,120.432530,87.672464,122.960142,80.008696,23,23,31
63,2021,4,2679.000000,71.333333,160762.373667,110.314000,30.873750,81.261906,185.530000,40.316667,...,66.760284,142.181426,252.474165,133.637195,74.968182,109.424815,46.463636,22,22,30
64,2021,5,2483.148387,71.354839,167652.750194,113.129583,34.269167,84.040032,162.409677,48.145161,...,90.791796,145.617597,285.751219,130.257703,81.076190,87.254511,51.383333,21,21,31
65,2021,6,2393.520000,68.933333,257923.627300,158.559000,52.754167,122.022061,183.256667,33.226667,...,131.411501,197.932285,289.100821,114.964815,124.809091,90.660317,52.624242,22,22,30


In [17]:
monthly.to_csv(out + pref + '_monthly.csv')

In [18]:
monthly_filt=monthly.loc[monthly['year']<2020]
daily_filt=daily.reset_index().loc[daily.reset_index()['year']<2020]

In [19]:
calmonthly=daily_filt.groupby(['month']).mean().reset_index()
calmonthly=calmonthly.drop(['year','day','volume','standard_volume','off_volume','morning_volume','evening_volume'],1)
calmonthly.rename(columns={'value':'value_mean','volume':'volume_mean','price_weight_av': 'price_weight_mean','off_hour_count':'off_hour_mean','standard_hour_count':'standard_hour_mean','morning_hour_count':'morning_hour_mean','evening_hour_count':'evening_hour_mean'}, inplace=True)

calmonthly[['price_median', 'off_price_median','standard_price_median', 'morning_price_median', 'evening_price_median']]=daily_filt.groupby(['month']).median().reset_index()[['price_mean','off_price_mean','standard_price_mean', 'morning_price_mean', 'evening_price_mean']]
calmonthly[['price_stdev', 'off_price_stdev','standard_price_stdev', 'morning_price_stdev', 'evening_price_stdev']]=daily_filt.groupby(['month']).std().reset_index()[['price_mean','off_price_mean','standard_price_mean', 'morning_price_mean', 'evening_price_mean']]

calmonthly[['volume', 'off_volume','standard_volume', 'morning_volume', 'evening_volume']]=monthly_filt.groupby(['month']).mean().reset_index()[['volume','off_volume','standard_volume', 'morning_volume', 'evening_volume']]
calmonthly[['value', 'off_value','standard_value', 'morning_value', 'evening_value']]=monthly_filt.groupby(['month']).mean().reset_index()[['value','off_value','standard_value', 'morning_value', 'evening_value']]
calmonthly[['off_power','standard_power', 'morning_power', 'evening_power']]=monthly_filt.groupby(['month']).mean().reset_index()[['off_power','standard_power', 'morning_power', 'evening_power']]

calmonthly

,month,day_of_week,value_mean,price_weight_mean,price_max,price_min,price_mean,power_max,power_min,power_mean,...,volume,off_volume,standard_volume,morning_volume,evening_volume,value,off_power,standard_power,morning_power,evening_power
0,1,69.774194,171502.907589,46.435143,97.634677,33.016613,57.888038,302.916129,46.225000,163.002702,...,118793.300,82903.925,27816.775,5167.975,2904.625,5.316590e+06,242.101815,100.428751,76.189963,64.546072
1,2,70.902655,179877.796257,49.497615,103.178584,32.019912,59.788651,290.268142,39.270796,153.171114,...,103164.700,68570.075,26354.975,5379.275,2860.375,5.081548e+06,213.423308,105.429541,89.501071,70.977113
2,3,71.354839,122263.680298,46.534632,98.810161,29.089435,54.396482,206.841935,41.371774,109.121035,...,79530.125,46726.150,23659.500,6207.825,2936.650,3.790174e+06,137.250930,85.935427,93.978394,66.365080
3,4,71.300000,147314.013033,48.019978,97.065000,27.759417,54.435606,211.252500,54.368333,127.286819,...,90134.375,48182.600,29595.050,8184.400,4172.325,4.419420e+06,142.302307,112.955699,129.062256,99.102018
4,5,68.870968,122180.921048,47.402281,111.561774,29.680887,58.650228,206.340323,36.551613,104.254852,...,75960.800,47638.600,20351.700,5538.400,2432.100,3.787609e+06,141.988073,72.604709,80.384404,52.917070
5,6,71.933333,214428.840700,60.067389,135.855750,37.021833,73.814376,292.988333,43.570000,146.128153,...,103545.375,67866.175,23696.300,6554.725,5428.175,6.432865e+06,203.592015,95.798965,104.131212,86.487967
6,7,70.451613,288041.045274,65.938480,145.048952,40.198952,79.237466,343.052419,56.512903,181.256203,...,131747.975,83105.625,32715.200,8958.050,6969.100,8.929272e+06,243.270595,128.614262,135.359012,104.837581
7,8,69.548387,317114.447613,63.705721,146.385161,37.352097,78.516558,404.115323,67.423387,212.489241,...,155054.725,96669.675,40083.425,9914.150,8387.475,9.830548e+06,292.509317,153.257825,145.450033,122.571492
8,9,72.200000,326803.601000,53.153377,119.131000,33.530500,64.384776,462.555833,107.923333,261.432229,...,185463.450,116762.100,51069.725,11650.225,5981.400,9.804108e+06,343.465430,195.832683,187.739291,145.598001
9,10,69.096774,349868.722105,49.904891,105.153306,31.895081,57.055690,494.072581,116.862097,304.698374,...,221635.675,133925.700,69330.775,13605.925,4773.275,1.084593e+07,391.821032,250.946279,200.986802,106.925469


In [20]:
calmonthly.to_csv(out + pref + '_calmonthly.csv')

In [21]:
calmonthly_hours=hourly.groupby(['month','hour']).mean()
calmonthly_hours=calmonthly_hours.drop(['year','day','day_of_week'],1)
calmonthly_hours

price      volume         value
month hour                                      
1     0      38.766790  232.988889   8892.753006
      1      38.007284  233.802469   8848.836833
      2      37.656049  233.528395   8803.654309
      3      37.552160  236.059877   8899.353438
      4      37.673172  238.518280   9002.655156
...                ...         ...           ...
12    19    130.661613   80.230968   8684.619781
      20     89.013548  105.837419   7205.543774
      21     73.281871  171.339355  10649.464465
      22     39.710839  254.710968   8996.688987
      23     37.374000  260.658710   9130.851213

[288 rows x 3 columns]

In [22]:
calmonthly_hours.to_csv(out + pref + '_calmonthly_hours.csv')